In [88]:
from flask import Flask, render_template, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import io
import base64

In [89]:

app = Flask(__name__)


In [90]:

# Load the VGG16 model
model = load_model('vgg16_model.keras')


In [91]:

@app.route('/')
def index():
    return render_template('index.html')


In [92]:

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the uploaded image file
        file = request.files['file']

        # Convert the FileStorage object to bytes using io.BytesIO
        img_bytes = io.BytesIO(file.read())

        # Load and preprocess the image
        img = image.load_img(img_bytes, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Make predictions
        predictions = model.predict(img_array)
        decoded_predictions = decode_predictions(predictions, top=3)[0]

        # Format predictions as JSON
        results = [{'label': label, 'score': float(score)} for (_, label, score) in decoded_predictions]

        # Encode the image as base64
        img_base64 = base64.b64encode(img_bytes.getvalue()).decode('utf-8')

        # Pass data to the HTML template
        return render_template('result.html', img_base64=img_base64, prediction=decoded_predictions[0])

    except Exception as e:
        return jsonify({'error': f'An error occurred: {str(e)}'}), 500


In [93]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Dec/2023 23:35:04] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 1s 748ms/step


127.0.0.1 - - [23/Dec/2023 23:35:12] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2023 23:35:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2023 23:35:17] "POST /predict HTTP/1.1" 500 -


1/1 [==============================] - 0s 479ms/step


127.0.0.1 - - [23/Dec/2023 23:35:32] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2023 23:35:34] "GET / HTTP/1.1" 200 -
